# Introduction

Cancer is a deadly disease, which is responsible for over nine million death in 2018 \cite{wth}. Therefore, it is crucial to find effective and efficient treatments. 
One of the most effective cancer treatments is radiotherapy, where cancer cells are killed using doses of radiation. 
However, the irradiation process not only kills cancer tumors but also affects healthy tissues surrounding the cancer tumor. 
Thus, accuracy in radiotherapy has to be increased to minimize the radiation dose delivered to healthy cells and maximize the dose to cancer tumors. 
If all of the radiotherapy steps are linked in a chain, tumor delineation is the weakest link, and its accuracy significantly impacts radiotherapy accuracy \cite{Gudi2017}\cite{Njeh}. Therefore, increasing the accuracy of tumor delineation is one of the challenges of radiotherapy treatment. 
Furthermore, a study conducted by \citet{Weiss2003} shows that due to interobserver variability, when different radiologists delineate the same case, the variation of gravity centers of these tumors is up to 0.6-0.7cm. 
When analyzing the result of interobserver, the uncertainty in delineation is even larger than patient positioning and organ motion \cite{Rusten2017}\cite{Weiss2003}. Therefore, one method of improving the radiotherapy accuracy is having more than one radiologist in one case. However, the long waiting time to delineate tumor (4 hours for a trained radiologist) \cite{Harari2010} makes this method almost impossible.

However, with the increasing of technology, tumor delineation can now be done automatically using deep learning, to be specific, using a convolutional neural network (CNN). The automatic delineation results can be used as one "observer" in radiotherapy.

In order to delineate tumor like a radiologist, a deep learning model has to be created. This process contains repeating a group of actions such as model training (learning from radiologists), model testing, and model modification until the best model, which performs most similar to the radiologist, is found. The author calls this group of actions an experiment. Currently, there are many software libraries that help to create a deep learning model. However, these libraries are not specialized for CNN in medical images. Besides, creating and running experiments have to be done manually, which consumes time and effort.

The goal of this project in DAT390 course is to create a Software Requirement Specification and a Software Design to develop a framework which automatically delineates cancer tumor as well as resolve the disadvantage of the existing deep learning libraries. The developed framework should satisfy the minimum requirements, in which users can use the resulted framework to perform a single experiment to delineate cancer tumors automatically.

This report will define a software requirement specification (SRS) and Design Document of a Keras-based framework for the automatic delineation of cancer tumors. Moreover, this report also includes the progress of the development based on the SRS and Design and the current results.

# Theory and Definition

## Convolutional Neural Network

Artificial Intelligence (AI) refers to the term of giving the machines human "brain" so that it can "think" as well as "act" like a human. Machine learning (ML), a subfield of AI, focuses on making predictions based on existing data. Deep learning is one of an approach in machine learning where the data is learned through layers of a neural network. In the context of automatic tumour delineation, AI means making a computer program to perform radiologists' jobs. In contrast, ML means analyzing each pixel in medical images and deciding if it belongs to a cancer tumor. Deep learning refers to one of the approaches of predicting cancer tumors by transforming the image data through a number of layers in a neural network.

Figure \ref{fig:nn} illustrates how a neural network learns. If we apply this figure to the current context, *Input X* refers to the set of medical images to be delineated. *True targets Y* refers to the delineation by a real radiologist. *Predictions Y'* refers to the delineation made by the neural network. The images data (*Input X*) go through each *layer* and transform by applying some *weights* in that layer. The outputs of each layer may be used as the inputs of another layer. This creates a network of layers. In many cases, an activation function is applied to the output of the layer. The output of the final layers is the *Predictions Y'*. By using the *Loss score* calculated by applying a *Loss function* on *Predictions Y'* and *True targets Y*, the *Optimizer* updates the *weights*. The goal of this learning process is to find the weights that minimize the *Loss score* so that the neural network can make predictions closest to the radiologist.

\begin{figure*}
  \includegraphics{img/neural_network.png}
  \caption{The Learning Process of a Neural Network. The Input X goes through some data transformation layers by applying some weights in each layers. After going through all layers, the final transformed data, called Predictions Y', combines with the True targets Y in a loss function to calculate the loss score. The optimizer uses this score to update the weights until the neural network finds the weights that minimises the Loss score. Orignated from F. Chollet. Deep learning with Python \cite{Chollet} }
  \label{fig:nn}
\end{figure*}

Convolutional Neural Network (CNN) is the type of neural network containing layers using a convolution filter to transform the images data. There are many types of convolution filters with different effects on the image data, either dilation, erosion, shrinkage, expansion, etc.  

## Sequential Architecture

Architecture refers to the structure of the layers in the CNN. It determines how the layers are connected in the neural network. In the CNN with sequential architecture, the layers are connected sequentially. That means there are only one input and one output for each layer, and the output of the preceding layer is the input of the next layer. We can imagine the sequential CNN as a stack of layers, where the input will penetrate through all these layers in a straight line.

## U-Net Architecture

In the U-net Architecture, the image data go through two paths: the downsampling (contraction) path and the upsampling (expansion) path. In the original paper, the U-net architecture is defined in figure \ref{fig:unet}

\begin{figure*}
  \includegraphics{img/unet.png}
  \caption{U-net architecture (example for 32x32 pixels in the lowest resolution). Each blue box corresponds to a multi-channel feature map. The number of channels is denoted on top of the box. The x-y-size is provided at the lower left edge of the box. White boxes represent copied feature maps. The arrows denote the different operations \cite{Olaf}. Originate from Olaf Ronneberger et al \cite{Olaf}}
  \label{fig:unet}
\end{figure*}

## Deep Learning Experiment

Experiments of deep learning to find a good enough model for automatic delineation should follow these steps:

Step 1. Choose the metrics. This is the criteria to determine if a CNN has been trained well enough to use as an "observer".

Step 2. Prepare the input data. The input data here means the medical images to be delineated. Typically, the data is split into three sets:

 - Training data. This set of data is used for training the CNN.
 - Validation data. This set is used for evaluating the performance of the CNN and tuning hyper-parameters.
 - Test data. This set should be isolated from the other two sets. The performance the CNN is decided by evaluating this set, not the validation data set.

Step 3. Define architecture to use in the CNN.

Step 4. Choose hyper-parameters. The hyper-parameters are any mutable object in the CNN. This includes the loss function, optimizer, the type of convolution layer, the activation, etc.

Step 5. Train the model on the training data with the selected hyper-parameters.

Step 6. Evaluate the performance of the model on the validation data with the metric chosen in step 1.

Step 7. Repeat step 4 until the combination of hyper-parameters with the chosen architecture that makes the best model based on the chosen metric is found.

Step 8. Repeat step 3 until an architecture, together with a combination of hyper-parameters that create the best model is found.

Step 9. Use the best model to check the performance on test data.

An experiment refers to the process from step 1 to step 6.

# The Framework

The development of the resulted framework, named *deoxys*, has the goal of providing the users the ability to run multiple experiments of different CNN models and then choose the best model for final prediction. This framework should be specialized in deep-learning in medical images, especially in auto-delineation of cancer tumor. Because of that, it should integrate u-net architecture and image preprocessing modules, as well as logging tools and performance visualization tools when running experiments. These are the minimum requirements of the framework. It can be later extended with other types of architectures, preprocessors, automation, interactive verbose configuration, and visualization.

The development time, as well as maintenance time, will range from October 1st, 2019 until May 1st, 2020. The first milestone is on January 6th, 2020, with the goal of creating a framework that satisfies the minimum requirements, which will be defined in detail in the software requirement specification (see \ref{software-requirement-specification}).

## Software Requirement Specification

This part defines the requirement specification of the developing framework. Because of that, terms indicating future-tense such as "should", "shall", "will" as well as terms indicating ability such as "can", "may" will be used when describing framework.

In order to reach the goal of the development, the *deoxys* framework should satisfy all the requirements defined in User Requirement Specification (see \ref{user-requirement-specification}) and System Requirement Specification (see \ref{system-requirement-specification})

### User Requirement Specification

Users are referring to master students, Ph.D. candidates, researchers, and anyone who wants to use deep-learning on the automatic delineation of cancer tumors.
This framework is targeted to the users with basic programming knowledge, including the usage of JSON data structure, and with the knowledge of deep learning, especially in convolutional neural network. Basic programming knowledge is including but not limited to object-oriented programming in python, other python libraries such as matplotlib, Keras, h5py.

With the help of *deoxys*, users shall have the ability to perform multiple CNN experiments by creating configurable JSON files. Users can define their own sequential or u-net model with the choices of layers, loss functions, optimizers, metrics and many other hyper-parameters. In addition, users can choose how to split the data for training, validation and testing. Each experiment should include training the data, logging the performance and evaluation of the trained model on test data. All trained models can be saved to disk and loaded back for the continuation of training or any other purposes.

As a follow-up after running an experiment, users can also check the predicted outputs as delineated images in comparison with the original image and view the performance graphs of the trained model.

Users with advanced programming knowledge can also customize and create their custom model architecture, layers, activation functions, loss functions, optimizers, metrics, etc...

### Use cases

From the user requirement specification, the *deoxys* framework should support the following six use cases:

1. Create a model
1. Train a model
1. Save a trained model
1. Load a model from file(s)
1. Set up an experiment
1. Create and apply customized model objects to the model


#### Use case diagram

Figure \ref{fig:usecase} shows all the use cases and their interaction inside the framework. There are three main flows of the use cases:

- Setting up an experiment using configurations to run and evaluate that experiment. This starts with creating a model from the configuration, then setting up an experiment by training and evaluating the configured model.
- Loading and saving trained model from and to disk. 
- Creating customize objects / elements for the experiment. This includes: Layers, Activation functions, Loss functions, Optimizers, Callbacks.

\begin{figure*}[h!]
  \includegraphics[width=\textwidth]{img/use_case.png}
  \caption{Use Case Diagram}
  \label{fig:usecase}
\end{figure*}

#### Use case 1: Create a model

Every action of the user involves the use of the model. The model term in the *deoxys* framework refers to a group of three components. 
The first component is a convolutional neural network, which can be a sequential CNN or an U-net CNN, or even a customized CNN defined by the users. This CNN contains input shapes, layers, activation functions. We call this component the architecture of the model. 
The second component is the set of hyper-parameters of the neural network, which includes the optimizers, loss function, and metrics. 
The last component, called Data Reader, acts as a data provider, which feeds the data, medical images with delineation contour,  into the neural network for training and evaluation. This involves splitting up the data into training data, validation data, and test data, as well as preprocessing the data to make sure the data is suitable for training in the CNN.

#### Use case 2: Train a model

Since a model contains all the components needed for training, the training process can be performed directly after a model is created.

#### Use case 3: Save a trained model
After the model is trained, it is necessary to save that model for later use, either to bring the saved model to another location to perform automatic tumor delineation, or to continue training.

#### Use case 4: Load a model from file(s)
Because a model can be saved to files, users should be able to load it back to use it later.

#### Use case 5: Set up an experiment

Instead of training a model directly, users can set up an experiment from a created model, or a saved model. This includes training the model while logging the performance of the models on the training data and the validation data using the predefined metrics. Besides, users can configure checkpoints for saving models and making predictions on validation data while training. Visualization of performance and predictions on test data can also be performed when after running an experiment.

#### Use case 6: Create and apply customized model objects to the model

Model objects refer to the layers in the neural network, the activation functions, the optimizers, the loss functions, metrics, and any other components existing in a model. Since not all types of objects can be predefined, users with advanced programming knowledge should be able to define customized objects and apply it to their models.

### System Requirement Specification

The *deoxys* framework should have the following attributes: usability, reliability, flexibility, maintainability and portability.

#### Usability
The *deoxys* framework should be easy to install, learn and use. The expected training and learning time for a user to use this framework effectively should not take more than 40 hours. For this reason, this framework should have detailed documentation of the installation guide and usage of each class, function and property. It should also provide sample code snippets which can be applied to the defined use cases.

#### Reliability
The output generated when running code from *deoxys* framework should have the behaviors as documented. In addition, the unexpected error rate should be under 5% and at least 80% of code lines should have been tested before release.

#### Flexibility
Users should be able to customize and create new components to integrate with *deoxys* framework.

#### Maintainability
The *deoxys* framework should be easy to maintain. Therefore, it should be divided into separated modules. Moreover, all of the source code should follow the PEP8 coding convention. Also, this framework should log all actions in different versions and issues from the users.

Maintaining the framework includes fixing bugs, handling issues, updating and adding new features. The maintenance activities should last at least until May 2020.

#### Portability
The *deoxys* framework should work properly when the following hardware requirements and environment are satisfied:

- System memory: at least 8GB with GPU or 13GB without GPU
- Python version: at least 3.7

## Designs

### Overview

Before development, the designs of the framework have to be considered.

The first things to concern are the usability and maintainability of the framework. As stated, in the previous sessions, all source code shall follow PEP8 coding convention. Sphinx will be used as the tool of documentation. In addition, git is used as a tool to handle logging and version management. All source code should be available in [http://github.com/huynhngoc/deoxys](http://github.com/huynhngoc/deoxys).

Implementation of all layers and other components in convolutional neural networks within a three-month time is impossible. Therefore, Keras is used as a based library, as it contains implemented layers, activation functions, optimizers and other components in CNNs. Also, Keras is compatible with TensorFlow 1.x, 2.x, which is a powerful backend tool in deep-learning, as well as other backends such as Theano, etc.

The author suggests that the framework should have the following modules:

- Models: contains a wrapper of a Keras model. Other Keras objects such as optimizers, activation functions, etc are also included.
- Architecture loader. The loader should be able to create models from configurable JSON objects that contain the architecture of the model. 
- Data reader: Since the target of this framework are medical images, the input data often has a large size and usually cannot fit into the computer memory. In order to avoid out of memory errors, this module should contain a data generator that split image data into smaller batches that can fit into the memory when training the model.
- Experiment: The *deoxys* framework should be able to perform a single experiment and multiple experiments.

#### Structure diagram

Figure \ref{fig:structure} illustrates the structure of the *deoxys* framework.

\begin{figure*}[h!]
  \includegraphics[width=\textwidth]{img/project_structure.png}
  \caption{\textit{Deoxys} Structure Diagram}
  \label{fig:structure}
\end{figure*}

### Model Objects

These modules are the components creating a model. They are layers, loss functions, activation functions, metrics, optimizer and callbacks. Any customized objects created by the users will be added to this module at runtime.

### Model

Firstly, this module should be a wrapper of a Keras model. As a result, it should have methods of the Keras model, such as:

- `load`: loading models
- `save`: save models to files
- `fit`: fit a model with data
- `predict`: predict the target
- `evaluate`: evaluate the performance of the current state of the model

Secondly, it should have a Data Reader (see \ref{data-reader}) instance, which provided proper inputs for actions on the model.

Finally, by performing methods in Keras model using the inputs from the data reader, the model should have the following methods:

- `fit_train`: fit the training data
- `predict_val`: predict the validation data
- `predict_test`: predict the test data
- `evaluate_test`: evaluate the performance of the current state of the model on the test data

### Architecture Loader

This module should have a function to create a model from one of the predefined architecture. The predefined architectures are the sequential and the U-net. In the future, dense model should be implemented to be used as a predefined architecture. This module should be able to load a configurable JSON file to create a Keras model based on the configuration.

### Data Reader

The data reader module should provide input data for training and evaluating the model. The data reader should provide three sets of data: training data, validation data, test data. These three sets should be in the form of a python generator, which is wrapped into a Data Generator. Using a python generator is essential because medical image data usually has a large size, and may not be able to fit into the running environment's memory. Using a python generator will feed the model with a small part of the data and minimize the chance of getting out of memory error. The list of preprocessors to be applied to the data should be configurable.

#### HDF5 Data Reader

`h5` or `hdf5` is a file format that has the ability to store large dataset with compression and hierarchy, as well as meta-data. The main components of an HDF5 file are groups and datasets, where datasets are pieces of data that is stored in file while groups are containers of datasets.

The *deoxys* framework should have a HDF5 Data Reader, which is a Data Reader that process data from a hdf5 file. As a result, it should provide the three datasets: train, validation and test. Also, since an HDF5 file can be split into groups, the HDF5 Data Reader should provide an aid for configuring which groups of data to be in the three basic sets. It should be easy to configure different groups into different purposes for cross-validation. The suggested structure of hdf5 file to be used in the HDF5 Data Reader is to split the data into folds, where the users can configure which folds to be in the training set, or the validation set, or the test set.

Here is an example of the structure of a hdf5 file to be used in the HDF5 Data Reader, `\fold_[n]` is the name of the group, and `col_[n]` are names of the datasets, each of which is the column data.
```
\fold_0
   col_0
   col_1
   col_2
\fold_1
   col_0
   col_1
   col_2
\fold_2
   col_0
   col_1
   col_2
\fold_3
   col_0
   col_1
   col_2
\fold_4
   col_0
   col_1
   col_2

```

With this HDF5 file with the example structure, the HDF5 Data Reader should allow users to configure the following five things:

- Path to the HDF5 file
- The column to be used as *Input X*
- The column to be used as *True Target Y*
- The maximum number of items provided by the data generator. We call this number the `batch_size`.
- Which folds belong to which set. For example, users can configure `fold_0` and `fold_1` to be used for training, while `fold_2` is for validation and `fold_3` and `fold_4` are for testing. In another experiment, users can configure `fold_1` and `fold_2` to be used for training, while `fold_0`  is for validation. In this way, users can use cross-validation in the framework.

### Experiment

#### Single Experiment
The Single Experiment is used to perform an experiment. With the use of Keras 'callbacks', the modules can have the following actions while training:

- log training performance
- log validation performance
- save a model to disk at a checkpoint
- use the model at the checkpoint to predict validation data

By using the files created during training, the Single Experiment can visualize the training and validation performance, as well as visualize the predictions. The visualization of predictions can be either tumor delineation by radiologists and by the model directly on the original images, or a plot containing three images: the original images, the ground truth masks from radiologists, and the predictions of the model. Moreover, it can find the best model of each metric based on the log files.

#### Multiple Experiment
The Multiple Experiment class should be able to run multiple single experiments, either concurrently or not. After finish all experiments, it should find the best model from all experiments and use that model to predict and evaluate performance on the test set.

# Results

## Implementation progress

### Completed modules

By the time this report is submitted, users can perform a single experiment, with saving, loading, and visualization using the *deoxys* framework. This means all parts but "multiple experiments" from the design diagram in figure \ref{fig:structure} have been implemented.

### In-progress modules
Modules related to running multiple experiments are still in development. There are problems involving the process of combining multiple single experiments into a batch of experiments, as well as the concurrent programming that allows running multiple experiments in parallel.

Besides, there is still a lack of tests and documentation that needs to be resolved.

## Run on test data

The data from Oslo University hospital was used for running a test experiment. It contained the CT and PET images to detect head and neck cancer. The model parameters were taken from Yngve Mardal Moe's master thesis \cite{yngve} and run the training set with only 3000 slices of images and three epochs. A criterion of success is that the trained model has the performance of f-beta score above 0.5.

The result was amazing as the dice (f-beta score) was about 0.5 (figure \ref{fig:dice}) and some samples had nice delineation results (figure \ref{fig:result})

\begin{figure}
  \includegraphics{img/binary_fbeta.png}
  \caption{Visualization of performance (binary f-beta score) of an experiment}
  \label{fig:dice}
\end{figure}

\begin{figure}
  \includegraphics{img/00028.png}
  \caption{Visualization of predictions of a sample in an experiment}
  \label{fig:result}
\end{figure}

# Discussions

As there are some meaningful results from the development, I can use this framework for running experiments in the Master's thesis.
There is plenty of space for improvement. Possible improvements are:

- More customized preprocessors and callbacks should be added to the framework.
- Development of an auto-generated configuration tool, either web-based or verbose terminal tool.
- Back-propagation implementation based on the implemented model. This will be developed in my Master's thesis.
- Visualize the progress of training/prediction.
- Data generator as a sequential model for multiprocessing

# Conclusion

By the time of creating this report, the development of the *deoxys* framework has satisfied the minimum requirements for running a single experiment. Users who are interested in automatic tumor delineation using deep-learning can try to create a model and run the experiment. The current development of *deoxys* can continue with many extensions and upgrades to create a more user-friendly framework with more features.